## Reinforcement Learning Models

### Imports and Utils

In [1]:
"""
Importing necessary libraries
"""
import pygame
import neat
import os
import numpy as np
import time

import pickle

pygame 2.4.0 (SDL 2.26.4, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
"""
Defining Utilities for the Game
"""
scale = 0.5
WIN_WIDTH = 1920 * scale
WIN_HEIGHT = 1080 * scale
CAR_WIDTH = 60 * scale
CAR_HEIGHT = 60 * scale

CAR_IMG = pygame.transform.scale(pygame.image.load(os.path.join("Images", "car.png")), (CAR_WIDTH, CAR_HEIGHT))
ROAD_IMGS = []
for i in range(1, 6):
    ROAD_IMGS.append(pygame.transform.scale(pygame.image.load(os.path.join("Images", f"map{i}.png")), (int(WIN_WIDTH), int(WIN_HEIGHT))))
BORDER_COLOR = (255, 255, 255, 255)

In [3]:
"""
Defining Car Class
"""
class Car:
    IMGS = CAR_IMG
    ROT_VEL = 5

    def __init__(self, position):
        self.position = position
        self.tilt = 0
        self.vel = 0

        self.image = self.IMGS
        self.center = (self.position[0] + self.image.get_width()//2, self.position[1] + self.image.get_height()//2)
        self.radars = []
        self.beams = []
        self.thetas = [90, 45, 0, -45, -90]

        self.dist = 0

    def engine(self, action):
        if action[0] == 1:
            self.vel += 2
            self.vel = min(self.vel, 10)
        if action[1] == 1:
            self.vel -= 2
            self.vel = max(self.vel, 0)
        if action[2] == 1:
            self.tilt += self.ROT_VEL
            self.tilt = self.tilt % 360
        if action[3] == 1:
            self.tilt -= self.ROT_VEL
            self.tilt = self.tilt % 360

    def move(self):
        self.dist += self.vel
        self.position[0] += self.vel*np.cos(np.radians(self.tilt))
        self.position[1] -= self.vel*np.sin(np.radians(self.tilt))
        self.center = (self.position[0] + self.image.get_width()//2, self.position[1] + self.image.get_height()//2)
    
    def draw(self, window):
        rotated_image = pygame.transform.rotate(self.image, self.tilt)
        new_rect = rotated_image.get_rect(center=self.image.get_rect(topleft=self.position).center)
        window.blit(rotated_image, new_rect.topleft)

    def get_radars(self, map):
        self.radars = []
        self.beams = []
        for i in self.thetas:
            length = 0
            x = int(self.center[0] + length*np.cos(np.radians(self.tilt + i)))
            y = int(self.center[1] - length*np.sin(np.radians(self.tilt + i)))
            while not map.get_at((x, y)) == BORDER_COLOR and length < 100:
                length += 1
                x = int(self.center[0] + length*np.cos(np.radians(self.tilt + i)))
                y = int(self.center[1] - length*np.sin(np.radians(self.tilt + i)))
            self.radars.append((x, y))
            self.beams.append(length)

    def get_mask(self):
        return pygame.mask.from_surface(self.image)
    


In [4]:
class Map:
    IMGS = ROAD_IMGS

    def __init__(self, road, position = [0, 0]):
        self.position = position
        self.image = self.IMGS[road]

        self.map_mask = self.get_mask()

    def collide(self, car):
        car_mask = car.get_mask()

        offset = (car.position[0] - self.position[0], car.position[1] - self.position[1])
        overlap = self.map_mask.overlap(car_mask, offset)

        return overlap is not None

    def get_mask(self):
        return pygame.mask.from_threshold(self.image, (255, 255, 255), (10, 10, 10, 255))     

In [5]:
"""
Game Structure and Functionality
"""
pygame.font.init()
RESTART_X = 830 * scale
RESTART_Y = 920 * scale
MAP_IND = 0

class RacingCarGame:

    def __init__(self, window, car, map):
        self.window = window
        self.car = car
        self.map = map

        self.clock = pygame.time.Clock()
        self.running = True
        self.score = 0

    def gameplay(self):
        game_over = False
        while self.running:
            self.clock.tick(30)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                if event.type == pygame.KEYDOWN:
                    if game_over and event.key == pygame.K_SPACE:
                        self.car = Car([RESTART_X, RESTART_Y])
                        self.map = Map(MAP_IND, [0, 0])
                        self.score = 0
                        game_over = False
                    if event.key == pygame.K_w:
                        self.car.engine([1, 0, 0, 0])
                    if event.key == pygame.K_s:
                        self.car.engine([0, 1, 0, 0])
                    if event.key == pygame.K_a:
                        self.car.engine([0, 0, 1, 0])
                    if event.key == pygame.K_d:
                        self.car.engine([0, 0, 0, 1])

            if not game_over:
                self.car.move()
                self.score = self.car.dist // 100
                if self.map.collide(self.car):
                    game_over = True

            if game_over:
                self.Game_Over()
            else:
                self.display(DRAW_LINES = True)
        pygame.quit()

    def display(self, DRAW_LINES = False):
        self.window.blit(self.map.image, (0, 0))
        self.car.draw(self.window)

        if DRAW_LINES:
            self.car.get_radars(self.map.image)
            for radar in self.car.radars:
                pygame.draw.line(self.window, (0, 255, 0), self.car.center, radar, 2)
                pygame.draw.circle(self.window, (0, 255, 0), radar, 5)

        text = pygame.font.SysFont("norwester", 25).render(f"Score: {self.score}", 1, (0, 0, 0))
        self.window.blit(text, (10, 10))
        pygame.display.update()

    def Game_Over(self):
        self.window.blit(self.map.image, (0, 0))
        self.car.draw(self.window)
        text = pygame.font.SysFont("norwester", 25).render(f"Score: {self.score}", 1, (0, 0, 0))
        self.window.blit(text, (10, 10))
        text = pygame.font.SysFont("norwester", 50).render("Game Over", 1, (0, 0, 0))
        self.window.blit(text, (WIN_WIDTH//2 - text.get_width()//2, WIN_HEIGHT//2 - text.get_height()//2))
        pygame.display.update()

### Defining Game Structure

In [12]:
def main():
    car = Car([RESTART_X, RESTART_Y])
    map = Map(MAP_IND, [0, 0])
    window = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))

    pygame.font.init()
    game = RacingCarGame(window, car, map)
    game.gameplay()

main()

### Model Creation and Training

In [6]:
class CompleteGenome(Exception):
    def __init__(self, genome, net, score):
        self.genome = genome
        self.net = net
        self.score = score

In [7]:
"""
Game Structure and Functionality
"""
pygame.font.init()
RESTART_X = 830 * scale
RESTART_Y = 920 * scale

class RacingCarGameAI:
    THRESHOLD = 100

    def __init__(self, window, cars, map, genes, nets):
        self.window = window
        self.map = map

        self.cars = cars
        self.genes = genes
        self.nets = nets

        self.clock = pygame.time.Clock()
        self.running = True
        self.gen = 0
        self.scores = [0 for _ in range(len(self.cars))]
        self.is_alive = [True for _ in range(len(self.cars))]

    def gameplay(self):
        start = time.time()
        if_raise = False
        while self.running:
            self.clock.tick(60)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                    best_ind = np.argmax(self.scores)
                    if_raise = True
                    break

            if np.max(self.scores) > self.THRESHOLD:
                self.running = False

                NA_inds = [i for i, alive in enumerate(self.is_alive) if not alive]

                for i in reversed(NA_inds):
                    self.cars.pop(i)
                    self.genes.pop(i)
                    self.nets.pop(i)
                    self.scores.pop(i)
                    self.is_alive = self.is_alive[:i] + self.is_alive[i+1:]
                break
                
            if not np.sum(self.is_alive):
                self.running = False
                break
                
            for i, cars in enumerate(self.cars):
                if self.is_alive[i]:
                    cars.get_radars(self.map.image)
                    output = self.nets[i].activate(cars.beams)
                    output = [1 if x > 0.5 else 0 for x in output]
                    cars.engine(output)
                    cars.move()

                    if self.map.collide(cars):
                        self.genes[i].fitness -= 1
                        self.is_alive[i] = False
                        continue

                    self.scores[i] = cars.dist // 100
                    self.genes[i].fitness = self.scores[i] + cars.vel

                    if time.time() - start > 2 and (cars.vel < 2 or cars.dist < 100):
                        self.cars.pop(i)
                        self.genes.pop(i)
                        self.nets.pop(i)
                        self.scores.pop(i)
                        self.is_alive = self.is_alive[:i] + self.is_alive[i+1:]

            self.display(DRAW_LINES = True)

        pygame.quit()
        if if_raise:
            raise CompleteGenome(self.genes[best_ind], self.nets[best_ind], self.scores[best_ind])

    def display(self, DRAW_LINES = False):
        self.window.blit(self.map.image, (0, 0))
        for car in self.cars:
            car.draw(self.window)

        if DRAW_LINES and np.sum(self.is_alive) > 0:
            for i, car in enumerate(self.cars):
                if self.is_alive[i]:
                    car.get_radars(self.map.image)
                    for radar in car.radars:
                        pygame.draw.line(self.window, (0, 255, 0), car.center, radar, 2)
                        pygame.draw.circle(self.window, (0, 255, 0), radar, 5)

        text = pygame.font.SysFont("norwester", 25).render(f"Generation: {self.gen}", 1, (0, 0, 0))
        self.window.blit(text, (10, 10))
        text = pygame.font.SysFont("norwester", 25).render(f"Max Score: {np.max(self.scores)}", 1, (0, 0, 0))
        self.window.blit(text, (10, 40))
        text = pygame.font.SysFont("norwester", 25).render(f"Alive: {np.sum(self.is_alive)}", 1, (0, 0, 0))
        self.window.blit(text, (10, 70))
        
        pygame.display.update()

In [8]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "Configs/config_RCAI.txt")
config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

population = neat.Population(config)
population.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
population.add_reporter(stats)
generation = 0

def main(genomes, config):
    global generation

    cars = []
    genes = []
    nets = []

    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        cars.append(Car([RESTART_X, RESTART_Y]))
        g.fitness = 0
        genes.append(g)

    road = Map(MAP_IND, [0, 0])
    window = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))

    try:
        pygame.font.init()
        game = RacingCarGameAI(window, cars, road, genes, nets)
        game.gen = generation
        game.gameplay()
    except CompleteGenome as e:
        raise e

    generation += 1

try:
    winner = population.run(main, 50)
except CompleteGenome as e:
    winner = e.genome

with open("Models/RacingCarAI.pkl", "wb") as genome_file:
    pickle.dump(winner, genome_file)
print(f"The Best Genome:\n{winner}")


 ****** Running generation 0 ****** 

Population's average fitness: 2.40000 stdev: 3.91663
Best fitness: 11.00000 - size: (4, 20) - species 1 - id 4
Average adjusted fitness: 0.218
Mean genetic distance 1.342, standard deviation 0.243
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    20     11.0    0.218     0
Total extinctions: 0
Generation time: 2.214 sec

 ****** Running generation 1 ****** 

Population's average fitness: 5.10000 stdev: 4.53762
Best fitness: 11.00000 - size: (4, 20) - species 1 - id 4
Average adjusted fitness: 0.464
Mean genetic distance 1.306, standard deviation 0.267
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    20     11.0    0.464     1
Total extinctions: 0
Generation time: 2.144 sec (2.179 average)

 ****** Running generation 2 ****** 

Population's average fitness: 5.70000 stdev: 4.

### Genome Loading

In [26]:
with open("Models/RacingCarAI.pkl", "rb") as genome_file:
    genome = pickle.load(genome_file)

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "Configs/config_RCAI.txt")
config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

model = neat.nn.FeedForwardNetwork.create(genome, config)

In [32]:
model.activate([1, 0, 10, 0, 0])

[-0.9999999999999553, 1.0, 0.9999999610438894, 0.9999796043932564]